In [7]:
!pip install google-ngram-downloader

  Stored in directory: /root/.cache/pip/wheels/8f/2f/58/b276b0d5c6aa83128500ff40af60142ffdc73d649e3892ccd2
  Stored in directory: /root/.cache/pip/wheels/96/c9/9a/249e1a1b257e4de5851902455374637c4d68af63e933803569
Successfully built google-ngram-downloader opster


Import data from https://github.com/EdinburghNLP/code-docstring-corpus


In [0]:
from nltk.util import everygrams
import nltk
from nltk.tokenize.simple import CharTokenizer
from collections import Counter
import itertools
from tqdm import tqdm, tqdm_notebook
import string

In [0]:
from google_ngram_downloader import readline_google_store

goog_ngram_counts = Counter()


ngram_generators = []
# for each n
for n in range(1,5):
    upstream_ngram_generators_generator = readline_google_store(ngram_len=n)
    for generator in upstream_ngram_generators_generator:
        fname, url, records = generator
        ngram_generators.append(records)
print(ngram_generators)

for gen in ngram_generators:
    for record in gen:
        print(record)

# i = 0
# while i < 20000:
#     record = next(records)
#     #print(record)
#     # skip ngrams with tags in them
#     # see https://books.google.com/ngrams/info
#     if record.match_count < 6000 or "_" in record.ngram:
#         continue
#     else:
#         i += 1
#         if i > 5000:
#             print("key ", record.ngram)
#             print("count ", record.match_count)
           

# print(fname)
# print(url)
# print(next(records))
# print(next(records))
# record

# Compute n-grams from English and Python

In [3]:
# dl Reuters english corpus
nltk.download("reuters")

[nltk_data] Downloading package reuters to /root/nltk_data...


True

In [4]:
char_stream = CharTokenizer().tokenize(" ".join(nltk.corpus.reuters.words()).lower())
print("number English characters in corpus: ", len(char_stream))

number English characters in corpus:  8607412


In [0]:
# everygram_counts = Counter(everygrams(char_stream, max_len=15))  # out of RAM lol time to chunk


bigass_iterator = everygrams(char_stream, max_len=15)

#https://stackoverflow.com/a/8998040/4212158
def grouper_it(n, iterable):  # split iterable into iterable of iterables
    it = iter(iterable)
    while True:
        chunk_it = itertools.islice(it, n)
        try:
            first_el = next(chunk_it)
        except StopIteration:
            return
        yield itertools.chain((first_el,), chunk_it)

everygram_counts = Counter()
for ngrams in grouper_it(100000, bigass_iterator):  # process everygrams in chunks of 100k
  chunk_ngram_counts = Counter(ngrams)
  for gram in chunk_ngram_counts.copy():
      # remove small instance words
      if chunk_ngram_counts[gram] <= 3:
        del chunk_ngram_counts[gram]
  everygram_counts += chunk_ngram_counts  # add to big list

In [6]:
print("total of {} everygrams".format(len(everygram_counts)))
top_tuples = everygram_counts.most_common(200)
for tup in top_tuples:
    # convert ('a', 'n', 'd' to 'and')
    print("<" + "".join([x for x in tup[0]]) + "> " + str(tup[1]))

total of 511807 everygrams
< > 1720900
<e> 716143
<t> 561269
<a> 522478
<s> 477753
<n> 465177
<i> 459254
<o> 457385
<r> 453146
<l> 291178
<d> 267469
<c> 235795
<s > 218828
<h> 218447
<e > 199450
<u> 172119
< t> 167140
<m> 165732
<p> 160060
<f> 142348
<t > 141721
<d > 139616
<n > 134288
< a> 130812
< s> 128985
<in> 124556
<th> 116850
<g> 112522
<.> 99809
<an> 98296
<he> 98212
<er> 98002
< .> 97586
<b> 96512
<re> 96248
<. > 95677
< . > 94687
< th> 93645
<y> 93552
< i> 92402
<r > 87786
<on> 84402
< o> 82179
< c> 82028
<w> 81967
<the> 81192
<,> 77632
< the> 75874
< ,> 75766
<he > 74832
<, > 74173
<v> 73038
< , > 72360
<es> 71773
<ar> 70609
<the > 69292
< the > 69277
< p> 67974
<y > 67526
<or> 64230
< b> 63988
< m> 63343
<0> 62811
<te> 62229
<to> 59864
<st> 59821
< f> 59135
<en> 57852
<at> 56710
<ed> 56517
<it> 56262
< in> 55307
<o > 55129
<co> 54090
<ti> 52952
<al> 52538
< d> 52370
<nd> 52218
< w> 51804
<1> 51589
<l > 50151
<nt> 50060
<ed > 49020
<of> 47862
< to> 46376
< r> 45850
<ro> 4496

# superset of chars

does not include any captial letters tho

In [0]:

# print(string.printable)
# print(len(string.printable))

everygram_superset = set()
top_tuples = everygram_counts.most_common(5000)
for tup in top_tuples:
  for char in tup[0]:
    everygram_superset.add(char)
  
#print(sorted(everygram_superset))

missing = set(string.printable) - everygram_superset
print(sorted(missing))

['\t', '\n', '\x0b', '\x0c', '\r', '!', '#', '$', '%', '*', '+', '<', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
